In [2]:
# data to generate tables for paper

import os
import pickle
import pandas as pd
import numpy as np

# Load the model data
with open("../models/m1_ordinallogit.pickle", "rb") as f:            
    m1_ordinallogit = pickle.load(f)

with open("../models/m2_unimodallogit.pickle", "rb") as f:            
    m2_unimodallogit = pickle.load(f)

with open("../models/m3_zerotrunc_unimodallogit.pickle", "rb") as f:            
    m3_zerotrunc_unimodallogit = pickle.load(f)

In [3]:
# Read the data
df = pd.read_csv('../dataset/tempe_cleaneddata.csv', sep='\t')

train_df = df.sample(frac=0.8, random_state=100)
test_df = pd.concat([df, train_df, train_df]).drop_duplicates(keep=False)

print(len(df))
print(len(train_df))
print(len(test_df))

39793
31834
7959


In [14]:
# Logistic CDF for ordered logit
def my_logisticcdf(val):
    return np.exp(val)/(1.0 + np.exp(val))

# Definitions for AMPCA and GMPCA
def AMPCA(proba, test_set, choice_col):
    sum = 0
    i = 0
    for sel_mode in test_set[choice_col].values:
        sum = sum + proba[i,sel_mode]
        i += 1
    N = i-1
    return sum/N

def CEL(proba, test_set, choice_col):
    sum = 0
    i = 0
    for sel_mode in test_set[choice_col].values:
        sum = sum + np.log(proba[i,sel_mode])
        i += 1
    N = i-1
    return -sum/N

def GMPCA(proba, test_set, choice_col):
    return np.exp(-CEL(proba, test_set, choice_col))

# Quadratic weighted kappa metric
def QWK(confusion_matrix, actual, predicted_vector, pred=None):
    N = len(confusion_matrix)
    print("confusion matrix:")
    print(pd.DataFrame(confusion_matrix).round(2))
    
    w = np.zeros((N, N))
    for i in range(len(w)):
        for j in range(len(w)):
            w[i][j] = float(((i-j)**2)/16)
    print('weights matrix:')
    print(w)
    
    act_hist=np.zeros([N])
    for item in actual:
        act_hist[item]+=1
    print("Actual histogram")
    print(act_hist)
    
    if pred is None:
        pred_hist = predicted_vector.sum(axis=0)
    print("Predicted histogram")
    print(pred_hist)
    
    E = np.outer(act_hist, pred_hist)
    E = E/E.sum() # normalize E
    print('Expected matrix (normalized):')
    print(pd.DataFrame(E))

    cm_norm = confusion_matrix/confusion_matrix.sum()
    print('Confusion matrix (normalized):')
    print(pd.DataFrame(cm_norm))
    
    num=0
    den=0
    for i in range(len(w)):
        for j in range(len(w)):
            num+=w[i][j]*cm_norm[i][j]
            den+=w[i][j]*E[i][j]

    weighted_kappa = (1 - (num/den))
    print('weighted kappa:')
    return weighted_kappa

## Model M1 Ordered Logit

In [5]:
# Create a dictionary of beta values
betas_m1 = {}
for b in m1_ordinallogit.betas:
    betas_m1[b.name] = b.value
    
print(betas_m1)

{'acc_leftturn_1': -0.27617923045487586, 'acc_rearend_1': -0.7845379569382471, 'acc_sides_1': -1.2943901110983305, 'action_lanes_1': -0.6178912372348411, 'action_slowing_1': -0.915160262195712, 'action_straight_1': -0.8298927414530792, 'action_turn_1': -1.0410497610536067, 'age_1': -0.008211583129379755, 'alcohol_1': 0.3838029859261252, 'cause_distraction_1': 0.08000954754860334, 'cause_following_1': -0.18542950656706514, 'cause_signal_1': -0.41198796030751295, 'cause_speeding_1': -0.026643784173297925, 'cause_turn_1': -0.15308938579397424, 'cause_unsafe_1': -0.49151970204516343, 'cause_yield_1': -0.10780087156236166, 'delta2': 2.610904132748593, 'delta3': 3.3104699884660373, 'delta4': 2.3027224685204026, 'hour_afternoon_1': -0.3633206951402312, 'hour_night_1': -0.33332950054437827, 'light_darklighted_1': -0.3474874878252927, 'light_darknotlighted_1': -0.663434559773627, 'meteo_cloudy_1': -0.17245722462309962, 'meteo_rain_1': -0.054008651225920835, 'nonintersection_1': -0.3157201912916

In [10]:
m1_results = test_df[['severity']].copy()

tau1 = betas_m1['tau1']
tau2 = tau1 + betas_m1['delta2']
tau3 = tau2 + betas_m1['delta3']
tau4 = tau3 + betas_m1['delta4']
    
for index, row in test_df.iterrows():
    # Define the utility
    U = 0
    for b in betas_m1.keys()-['tau1', 'delta2', 'delta3', 'delta4']:
        var_name = b[:-2] # removes "_2" at the end of label
        U += betas_m1[b]*row[var_name]

    if U<tau1:
        estimated = 0
    elif (U<tau2) & (U>=tau1):
        estimated = 1
    elif (U<tau3) & (U>=tau2):
        estimated = 2
    elif (U<tau4) & (U>=tau3):
        estimated = 3
    else:
        estimated = 4
    
    # Choice proba
    m1_results.loc[index, 'choice_proba_0'] = 1-my_logisticcdf(U-tau1)
    m1_results.loc[index, 'choice_proba_1'] = my_logisticcdf(U-tau1) - my_logisticcdf(U-tau2)
    m1_results.loc[index, 'choice_proba_2'] = my_logisticcdf(U-tau2) - my_logisticcdf(U-tau3)
    m1_results.loc[index, 'choice_proba_3'] = my_logisticcdf(U-tau3) - my_logisticcdf(U-tau4)
    m1_results.loc[index, 'choice_proba_4'] = my_logisticcdf(U-tau4)
    
    m1_results.loc[index, 'estimated_severity'] = estimated
    m1_results.loc[index, 'utility'] = U
    
m1_results.head()

,severity,choice_proba_0,choice_proba_1,choice_proba_2,choice_proba_3,choice_proba_4,estimated_severity,utility
9,0,0.659981,0.303549,0.035090,0.001241,0.000138,0.0,-0.663208
27,3,0.027689,0.251650,0.634602,0.076733,0.009327,2.0,3.558651
28,0,0.887971,0.102845,0.008846,0.000304,0.000034,0.0,-2.070185
40,0,0.814575,0.168976,0.015839,0.000549,0.000061,0.0,-1.480017
44,0,0.729717,0.243792,0.025499,0.000893,0.000099,0.0,-0.993187


In [12]:
# Confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(m1_results['severity'], m1_results['estimated_severity'])

array([[5478,   20,    0,    0,    0],
       [ 117,  870,  366,   28,   30],
       [  51,  472,  322,   42,   27],
       [   2,   62,   39,    5,    4],
       [   7,   14,    2,    0,    1]], dtype=int64)

In [16]:
# cm = pd.DataFrame(np.zeros((5,5)))
# for row in m1_results.loc[:,'severity':'choice_proba_4'].iterrows():
#     label = int(row[1]['severity'])
#     prob = row[1]['choice_proba_0':'choice_proba_4'].values
#     cm.iloc[label, :] = cm.iloc[label, :] + prob
# cm = pd.DataFrame(confusion_matrix(m1_results['severity'], m1_results['estimated_severity']))
# QWK(cm.values, m1_results['severity'].values, m1_results.loc[:,'choice_proba_0':'choice_proba_4'].values)

In [17]:
from sklearn.metrics import classification_report

target_names = ['No injury', 'Possible injury', 'Non-incapacitating (minor) injury', 'incapacitating (major) injury', 'fatal injury']
print(classification_report(m1_results['severity'], m1_results['estimated_severity'], target_names=target_names))

                                   precision    recall  f1-score   support

                        No injury       0.97      1.00      0.98      5498
                  Possible injury       0.61      0.62      0.61      1411
Non-incapacitating (minor) injury       0.44      0.35      0.39       914
    incapacitating (major) injury       0.07      0.04      0.05       112
                     fatal injury       0.02      0.04      0.02        24

                         accuracy                           0.84      7959
                        macro avg       0.42      0.41      0.41      7959
                     weighted avg       0.83      0.84      0.83      7959



In [18]:
from sklearn.metrics import cohen_kappa_score

dca = sum(m1_results['severity'] == m1_results['estimated_severity']) / len(m1_results)
proba = m1_results[['choice_proba_0', 'choice_proba_1', 'choice_proba_2', 'choice_proba_3', 'choice_proba_4']].to_numpy()
qwk = cohen_kappa_score(m1_results['severity'], m1_results['estimated_severity'], weights='quadratic')

## Discrete Classification Accuracy (DCA)
print("DCA: {dca:.4f}".format(dca=dca))

## Arithmetic Mean Probability of Correct Assignment (AMPCA)
print("AMPCA: {ampca:.4f}".format(ampca=AMPCA(proba, test_df, 'severity')))

## Geometric Mean Probability of Correct Assignment (GMPCA)
print("GMPCA: {gmpca:.4f}".format(gmpca=GMPCA(proba, test_df, 'severity')))

# Quadratic Weighted Kappa (QWK)
print("QWK: {qwk:.4f}".format(qwk=qwk))

DCA: 0.8388
AMPCA: 0.7206
GMPCA: 0.5812
QWK: 0.7582


## Model M2 Unimodal Ordinal Logit

In [19]:
# Create a dictionary of beta values
betas_m2 = {}
for b in m2_unimodallogit.betas:
    betas_m2[b.name] = b.value

In [20]:
m2_results = test_df[['severity']].copy()

for index, row in test_df.iterrows():
    # Define the utility
    f = 0
    for b in betas_m2.keys()-['ASC_possinjury', 'ASC_nonincap', 'ASC_incap', 'ASC_fatal']:
        var_name = b[:-2]
        f += betas_m2[b]*row[var_name]
    lmda = np.log(1+np.exp(f))
    
    # Utilities
    U = []
    U.append(0) # U_0
    U.append(betas_m2['ASC_possinjury'] + 2 * np.log(lmda) - lmda - np.log(1*2)) # U_1
    U.append(betas_m2['ASC_nonincap'] + 3 * np.log(lmda) - lmda - np.log(1*2*3)) # U_2
    U.append(betas_m2['ASC_incap'] + 4 * np.log(lmda) - lmda - np.log(1*2*3*4)) # U_3
    U.append(betas_m2['ASC_fatal'] + 5 * np.log(lmda) - lmda - np.log(1*2*3*4*5)) # U_4
    
    # Choice proba
    i = 0
    sum_U = np.sum(np.exp(U))
    for i in range(0, len(U)):
        #m2_results.loc[index, 'utility_'+str(i)] = U[i]
        m2_results.loc[index, 'choice_proba_'+str(i)] = np.exp(U[i])/sum_U
    
    # Estimate the severity of the accident (choosing the highest probability)
    estimated = 0
    if m2_results.loc[index, 'choice_proba_1'] > m2_results.loc[index, 'choice_proba_0']:
        estimated = 1
    if m2_results.loc[index, 'choice_proba_2'] > m2_results.loc[index, 'choice_proba_1']:
        estimated = 2
    if m2_results.loc[index, 'choice_proba_3'] > m2_results.loc[index, 'choice_proba_2']:
        estimated = 3
    if m2_results.loc[index, 'choice_proba_4'] > m2_results.loc[index, 'choice_proba_3']:
        estimated = 4
    m2_results.loc[index, 'estimated_severity'] = estimated
    
m2_results.head()

,severity,choice_proba_0,choice_proba_1,choice_proba_2,choice_proba_3,choice_proba_4,estimated_severity
9,0,0.500085,0.444042,0.055244,6.235603e-04,5.669393e-06,0.0
27,3,0.047863,0.265090,0.562139,1.081488e-01,1.675973e-02,2.0
28,0,0.980728,0.018958,0.000314,4.709391e-07,5.694855e-10,0.0
40,0,0.896824,0.099147,0.004015,1.475007e-05,4.364986e-08,0.0
44,0,0.875894,0.118736,0.005348,2.185031e-05,7.191596e-08,0.0


In [21]:
# Confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(m2_results['severity'], m2_results['estimated_severity'])

array([[5374,  123,    1,    0,    0],
       [  11,  921,  479,    0,    0],
       [  11,  494,  409,    0,    0],
       [   0,   62,   49,    0,    1],
       [   5,   14,    5,    0,    0]], dtype=int64)

In [22]:
# for row in m2_results.loc[:,'severity':'choice_proba_4'].iterrows():
#     label = int(row[1]['severity'])
#     prob = row[1]['choice_proba_0':'choice_proba_4'].values
#     cm.iloc[label, :] = cm.iloc[label, :] + prob
# cm = pd.DataFrame(confusion_matrix(m1_results['severity'], m1_results['estimated_severity']))
# QWK(cm.values, m2_results['severity'].values, m2_results.loc[:,'choice_proba_0':'choice_proba_4'].values)

In [23]:
from sklearn.metrics import classification_report

target_names = ['No injury', 'Possible injury', 'Non-incapacitating (minor) injury', 'incapacitating (major) injury', 'fatal injury']
print(classification_report(m2_results['severity'], m2_results['estimated_severity'], target_names=target_names))

                                   precision    recall  f1-score   support

                        No injury       1.00      0.98      0.99      5498
                  Possible injury       0.57      0.65      0.61      1411
Non-incapacitating (minor) injury       0.43      0.45      0.44       914
    incapacitating (major) injury       0.00      0.00      0.00       112
                     fatal injury       0.00      0.00      0.00        24

                         accuracy                           0.84      7959
                        macro avg       0.40      0.42      0.41      7959
                     weighted avg       0.84      0.84      0.84      7959



C:\Users\Melvin\AppData\Local\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Melvin\AppData\Local\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Melvin\AppData\Local\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
from sklearn.metrics import cohen_kappa_score

dca = sum(m2_results['severity'] == m2_results['estimated_severity']) / len(m2_results)
proba = m2_results[['choice_proba_0', 'choice_proba_1', 'choice_proba_2', 'choice_proba_3', 'choice_proba_4']].to_numpy()
qwk = cohen_kappa_score(m2_results['severity'], m2_results['estimated_severity'], weights='quadratic')

## Discrete Classification Accuracy (DCA)
print("DCA: {dca:.4f}".format(dca=dca))

## Arithmetic Mean Probability of Correct Assignment (AMPCA)
print("AMPCA: {ampca:.4f}".format(ampca=AMPCA(proba, test_df, 'severity')))

## Geometric Mean Probability of Correct Assignment (GMPCA)
print("GMPCA: {gmpca:.4f}".format(gmpca=GMPCA(proba, test_df, 'severity')))

# Quadratic Weighted Kappa (QWK)
print("QWK: {qwk:.4f}".format(qwk=qwk))

DCA: 0.8423
AMPCA: 0.7756
GMPCA: 0.6534
QWK: 0.8054


## Model M3 Zero-truncated Unimodal Ordinal Logit

In [26]:
# Create a dictionary of beta values
betas_m3 = {}
for b in m3_zerotrunc_unimodallogit.betas:
    betas_m3[b.name] = b.value

In [27]:
m3_results = test_df[['severity']].copy()

for index, row in test_df.iterrows():
    # Define the utility
    f = 0
    for b in betas_m3.keys()-['ASC_possinjury', 'ASC_nonincap', 'ASC_incap', 'ASC_fatal']:
        var_name = b[:-2]
        f += betas_m3[b]*row[var_name]
    lmda = np.log(1+np.exp(f))
    
    # Utilities
    U = []
    U.append(0) # U_0
    U.append(betas_m3['ASC_possinjury'] + 2 * np.log(lmda) - lmda - np.log(1*2) - np.log(1-np.exp(-lmda))) # U_1
    U.append(betas_m3['ASC_nonincap'] + 3 * np.log(lmda) - lmda - np.log(1*2*3) - np.log(1-np.exp(-lmda))) # U_2
    U.append(betas_m3['ASC_incap'] + 4 * np.log(lmda) - lmda - np.log(1*2*3*4) - np.log(1-np.exp(-lmda))) # U_3
    U.append(betas_m3['ASC_fatal'] + 5 * np.log(lmda) - lmda - np.log(1*2*3*4*5) - np.log(1-np.exp(-lmda))) # U_4
    
    # Choice proba
    i = 0
    sum_U = np.sum(np.exp(U))
    for i in range(0, len(U)):
        #m3_results.loc[index, 'utility_'+str(i)] = U[i]
        m3_results.loc[index, 'choice_proba_'+str(i)] = np.exp(U[i])/sum_U
    
    # Estimate the severity of the accident (choosing the highest probability)
    estimated = 0
    if m3_results.loc[index, 'choice_proba_1'] > m3_results.loc[index, 'choice_proba_0']:
        estimated = 1
    if m3_results.loc[index, 'choice_proba_2'] > m3_results.loc[index, 'choice_proba_1']:
        estimated = 2
    if m3_results.loc[index, 'choice_proba_3'] > m3_results.loc[index, 'choice_proba_2']:
        estimated = 3
    if m3_results.loc[index, 'choice_proba_4'] > m3_results.loc[index, 'choice_proba_3']:
        estimated = 4
    m3_results.loc[index, 'estimated_severity'] = estimated
    
m3_results

,severity,choice_proba_0,choice_proba_1,choice_proba_2,choice_proba_3,choice_proba_4,estimated_severity
9,0,0.421208,0.481905,0.095744,1.132197e-03,1.020603e-05,1.0
27,3,0.088188,0.180495,0.596427,1.173030e-01,1.758675e-02,2.0
28,0,0.933096,0.066171,0.000733,4.830613e-07,2.427305e-10,0.0
40,0,0.831020,0.163868,0.005103,9.458466e-06,1.336407e-08,0.0
44,0,0.684827,0.294635,0.020453,8.450347e-05,2.661486e-07,0.0
...,...,...,...,...,...,...,...
39760,1,0.247144,0.525850,0.221355,5.545936e-03,1.059220e-04,1.0
39777,0,0.983586,0.016371,0.000042,6.522044e-09,7.655759e-13,0.0
39782,1,0.207973,0.086758,0.494309,1.676273e-01,4.333275e-02,2.0
39787,0,0.856034,0.140338,0.003623,5.565777e-06,6.518619e-09,0.0


In [28]:
# Confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(m3_results['severity'], m3_results['estimated_severity'])

array([[5324,  172,    2,    0,    0],
       [  90,  784,  535,    2,    0],
       [  46,  403,  464,    1,    0],
       [   3,   52,   56,    0,    1],
       [   5,   10,    9,    0,    0]], dtype=int64)

In [29]:
# for row in m3_results.loc[:,'severity':'choice_proba_4'].iterrows():
#     label = int(row[1]['severity'])
#     prob = row[1]['choice_proba_0':'choice_proba_4'].values
#     cm.iloc[label, :] = cm.iloc[label, :] + prob
# cm = pd.DataFrame(confusion_matrix(m3_results['severity'], m3_results['estimated_severity']))
# QWK(cm.values, m3_results['severity'].values, m3_results.loc[:,'choice_proba_0':'choice_proba_4'].values)

In [30]:
from sklearn.metrics import classification_report

target_names = ['No injury', 'Possible injury', 'Non-incapacitating (minor) injury', 'incapacitating (major) injury', 'fatal injury']
print(classification_report(m3_results['severity'], m3_results['estimated_severity'], target_names=target_names))

                                   precision    recall  f1-score   support

                        No injury       0.97      0.97      0.97      5498
                  Possible injury       0.55      0.56      0.55      1411
Non-incapacitating (minor) injury       0.44      0.51      0.47       914
    incapacitating (major) injury       0.00      0.00      0.00       112
                     fatal injury       0.00      0.00      0.00        24

                         accuracy                           0.83      7959
                        macro avg       0.39      0.41      0.40      7959
                     weighted avg       0.82      0.83      0.82      7959



In [31]:
from sklearn.metrics import cohen_kappa_score

dca = sum(m3_results['severity'] == m3_results['estimated_severity']) / len(m3_results)
proba = m3_results[['choice_proba_0', 'choice_proba_1', 'choice_proba_2', 'choice_proba_3', 'choice_proba_4']].to_numpy()
qwk = cohen_kappa_score(m3_results['severity'], m3_results['estimated_severity'], weights='quadratic')

## Discrete Classification Accuracy (DCA)
print("DCA: {dca:.4f}".format(dca=dca))

## Arithmetic Mean Probability of Correct Assignment (AMPCA)
print("AMPCA: {ampca:.4f}".format(ampca=AMPCA(proba, test_df, 'severity')))

## Geometric Mean Probability of Correct Assignment (GMPCA)
print("GMPCA: {gmpca:.4f}".format(gmpca=GMPCA(proba, test_df, 'severity')))

# Quadratic Weighted Kappa (QWK)
print("QWK: {qwk:.4f}".format(qwk=qwk))

DCA: 0.8257
AMPCA: 0.7264
GMPCA: 0.5899
QWK: 0.7866
